<a href="https://colab.research.google.com/github/SridharSeshadri56/Decision_Models/blob/main/pyomoTransportationProblemDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import * works when algebraic formulation using sets. Else use the import as pyo

In [1]:
pip install pyomo  #Installs the modeling language called pyomo

     |████████████████████████████████| 9.6 MB 6.8 MB/s 
     |████████████████████████████████| 49 kB 841 kB/s 


In [2]:
# The GLPK (GNU Linear Programming Kit) package is intended for solving large-scale linear programming (LP), 
# mixed integer programming (MIP), and other related problems. It is a set of routines written in ANSI C and 
# organized in the form of a callable library.
!apt-get install -y -qq glpk-utils  #Installs the optimization engine called glpk.


Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [ ]:
from pyomo.environ import *
import itertools                          # package helps create products of sets. here I use to create product of demand and supply sets

demand_points = ['Ratkeller', 'McGoldricks', 'Night Train', 'Stern']    # This is the set of demand points
supply_points = ['Hoboken', 'Bronx', 'Brooklyn']                        # This is the set of supply points

demand = {'Ratkeller': 80, 'McGoldricks': 65, 'Night Train': 70, 'Stern': 85}          # demand at demand points 
supply = {'Hoboken': 80, 'Bronx': 145, 'Brooklyn': 120}                                # supply at supply points

flows_from_to_arcs = list(itertools.product(supply, demand))   # this creates a list with supply and demand point names
                                                               # this will become our set of variables
                                                               # we could loop over supply and demand and add to a list instead

cost_from_to = {('Hoboken','Ratkeller'):4.64, ('Hoboken','McGoldricks'):5.13, ('Hoboken','Night Train'): 6.54, ('Hoboken', 'Stern'):8.67, \
                ('Bronx','Ratkeller'):3.52, ('Bronx','McGoldricks'):4.16, ('Bronx','Night Train'): 6.90, ('Bronx', 'Stern'): 7.91, \
                ('Brooklyn','Ratkeller'):9.95, ('Brooklyn','McGoldricks'):6.82, ('Brooklyn','Night Train'): 3.88, ('Brooklyn', 'Stern'): 6.85 }       
                       # cost of ship one unit from- to

constraints = {'supply_constraint', 'demand_constraint'}           # The two sets of constraints

model = ConcreteModel(name = "(Model2)")                            # Same as previous
model.x = Var( flows_from_to_arcs, within= NonNegativeReals )       # Decision variables are the flows from - to
model.value = Objective(                                            # Objective
expr = sum( cost_from_to[i]*model.x[i] for i in flows_from_to_arcs), sense = minimize )  # Minimize total transportation cost

# This defines a rule called demand must be met. 

def demand_must_be_met_rule(m,c):
    return sum(m.x[i,c] for i in supply_points) == demand [c]    # sums supply to demand point c. Note the syntax == for saying equal to

# This defines a rule to make sure do not exceed supply constraints

def supply_must__not_be_exceeded_rule(m,c):
    return sum(m.x[c,i] for i in demand_points) <= supply [c]    # sums from supply point c to demand points.

# This defines in our model the constraints! Note that we simply pass the set of constraints and the rule. It does the rest.
# model is by default when we call (recall model can be renamed as you like )
model.demand_constraint = Constraint(demand_points, rule = demand_must_be_met_rule)   # applies to each demand point

model.supply_constraint = Constraint(supply_points, rule = supply_must__not_be_exceeded_rule)  # applies to each supply point

opt = SolverFactory('glpk')           # same as before

model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)   # same as before
results = opt.solve(model, tee= True)                 # same as before

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpgyuyvs7_.glpk.raw --wglp /tmp/tmpzm21pr98.glpk.glp --cpxlp
 /tmp/tmpb4tvvgxp.pyomo.lp
Reading problem data from '/tmp/tmpb4tvvgxp.pyomo.lp'...
8 rows, 13 columns, 25 non-zeros
81 lines were read
Writing problem data to '/tmp/tmpzm21pr98.glpk.glp'...
69 lines were written
GLPK Simplex Optimizer, v4.65
8 rows, 13 columns, 25 non-zeros
Preprocessing...
7 rows, 12 columns, 24 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7
      0: obj =   1.983300000e+03 inf =   1.150e+02 (1)
      3: obj =   1.899950000e+03 inf =   0.000e+00 (0)
*     8: obj =   1.469550000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (41789 bytes)
Writing basic solution to '/tmp/tmpgyuyvs7_.glpk.raw'...
30 lines were written


In [ ]:
model.pprint()

3 Set Declarations
    demand_constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Ratkeller', 'McGoldricks', 'Night Train', 'Stern'}
    supply_constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Hoboken', 'Bronx', 'Brooklyn'}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :   12 : {('Hoboken', 'Ratkeller'), ('Hoboken', 'McGoldricks'), ('Hoboken', 'Night Train'), ('Hoboken', 'Stern'), ('Bronx', 'Ratkeller'), ('Bronx', 'McGoldricks'), ('Bronx', 'Night Train'), ('Bronx', 'Stern'), ('Brooklyn', 'Ratkeller'), ('Brooklyn', 'McGoldricks'), ('Brooklyn', 'Night Train'), ('Brooklyn', 'Stern')}

1 Var Declarations
    x : Size=12, Index=x_index
        Key                         : Lower : Value : Upper : Fixed : Stale : Domain
    